In [1]:
import os
import uuid
import numpy as np
import pandas as pd
import geopandas as geopd

In [2]:
# modificar segun entorno local
os.chdir( "/Volumes/hd_mvf_datasets/henry_data")

# no modificar
folder_data = "1_external"
folder_pipeline = "2_pipeline"
folder_output = "3_output"

In [3]:
yelp_pa_javi1 = pd.read_csv(os.path.join(folder_pipeline, 'Philly_top20.csv'))
yelp_pa_javi2 = pd.read_csv(os.path.join(folder_pipeline, 'Philly_bottom20.csv'))
yelp_pa_javi = pd.concat([yelp_pa_javi1,yelp_pa_javi2]).rename(
        columns={
            "name":"perfil_nombre"
            ,"address":"perfil_direccion"
            #,"city":"county_name"
            #,"state":"state_usps"
            ,"stars":"perfil_calificacion_yelp"
            ,"categories":"perfil_tipo"
            }
    )
list_javi_business_id = yelp_pa_javi["business_id"].unique()
yelp_pa_javi.info()
yelp_pa_javi.head(1)


<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 0 to 19
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   business_id               40 non-null     object 
 1   perfil_nombre             40 non-null     object 
 2   perfil_direccion          40 non-null     object 
 3   latitude                  40 non-null     float64
 4   longitude                 40 non-null     float64
 5   perfil_calificacion_yelp  40 non-null     float64
 6   review_count              40 non-null     float64
 7   is_open                   40 non-null     int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 2.8+ KB


,business_id,perfil_nombre,perfil_direccion,latitude,longitude,perfil_calificacion_yelp,review_count,is_open
0,1w40T5J6br425rMvpS1zgQ,La Cocina del Cafe,1500 S Broad St,39.931674,-75.168458,5.0,24.0,1


In [4]:
n_size = int(4000)
n_clusters = int(n_size/8)
n_tipo = int(n_size/10)
data_random_powerbi = pd.DataFrame({
    "kpi_1_satisfaccion": np.random.normal(loc=60, scale=15.0, size=(n_size))
    ,"kpi_2_popularidad": np.random.normal(loc=60, scale=15.0, size=(n_size))
    ,"kpi_3_lealtad": np.random.normal(loc=60, scale=15.0, size=(n_size))
    ,"cluster":sum([["cluster_a"]*n_clusters,
                    ["cluster_b"]*n_clusters,
                    ["cluster_c"]*n_clusters,
                    ["cluster_d"]*n_clusters,
                    ["cluster_e"]*n_clusters,
                    ["cluster_f"]*n_clusters,
                    ["cluster_g"]*n_clusters,
                    ["cluster_h"]*n_clusters],[])
    ,"perfil_calificacion_yelp":np.random.normal(loc=60, scale=15.0, size=(n_size))
    ,"perfil_calificacion_gmaps":np.random.normal(loc=60, scale=15.0, size=(n_size))
    ,"perfil_nombre":np.nan
    ,"perfil_direccion":np.nan
    ,"perfil_tipo":["comida_a","comida_b","comida_c","comida_d","comida_e"]*2*n_tipo
    ,"perfil_otros":np.nan
    })
data_random_powerbi["kpi_4_indice"] = (
    (data_random_powerbi["kpi_1_satisfaccion"]*0.4) +
    (data_random_powerbi["kpi_2_popularidad"]*0.3) +
    (data_random_powerbi["kpi_3_lealtad"]*0.3)
    )

In [5]:
yelp_geodf_zcta_census = pd.read_pickle(
    os.path.join(folder_pipeline, 'data_zcta_varsxarea.pkl')).dropna().iloc[:,[0,1,2,17,18,19]]
yelp_geodf_zcta_census.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 165586 entries, 0 to 166416
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   business_id  165586 non-null  string 
 1   latitude     165586 non-null  Float64
 2   longitude    165586 non-null  Float64
 3   state_name   165586 non-null  object 
 4   state_usps   165586 non-null  object 
 5   county_name  165586 non-null  object 
dtypes: Float64(2), object(3), string(1)
memory usage: 9.2+ MB


In [6]:
yelp_geodf_zcta_census = yelp_geodf_zcta_census[~yelp_geodf_zcta_census.business_id.isin(list_javi_business_id)].copy()
n_sample = len(data_random_powerbi.index)
yelp_geodf_zcta_census_sample = yelp_geodf_zcta_census.sample(n=n_sample)
yelp_geodf_zcta_census_sample.reset_index(drop=True, inplace=True)
data_random_powerbi = data_random_powerbi.join(yelp_geodf_zcta_census_sample)
data_random_powerbi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   kpi_1_satisfaccion         4000 non-null   float64
 1   kpi_2_popularidad          4000 non-null   float64
 2   kpi_3_lealtad              4000 non-null   float64
 3   cluster                    4000 non-null   object 
 4   perfil_calificacion_yelp   4000 non-null   float64
 5   perfil_calificacion_gmaps  4000 non-null   float64
 6   perfil_nombre              0 non-null      float64
 7   perfil_direccion           0 non-null      float64
 8   perfil_tipo                4000 non-null   object 
 9   perfil_otros               0 non-null      float64
 10  kpi_4_indice               4000 non-null   float64
 11  business_id                4000 non-null   string 
 12  latitude                   4000 non-null   Float64
 13  longitude                  4000 non-null   Float

In [7]:
yelp_geodf_zcta_census_javi = pd.concat([data_random_powerbi,yelp_pa_javi])
yelp_geodf_zcta_census_javi.to_csv(
    os.path.join(folder_output, 'data_random_powerbi.csv'))
yelp_geodf_zcta_census_javi.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4040 entries, 0 to 19
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   kpi_1_satisfaccion         4000 non-null   float64
 1   kpi_2_popularidad          4000 non-null   float64
 2   kpi_3_lealtad              4000 non-null   float64
 3   cluster                    4000 non-null   object 
 4   perfil_calificacion_yelp   4040 non-null   float64
 5   perfil_calificacion_gmaps  4000 non-null   float64
 6   perfil_nombre              40 non-null     object 
 7   perfil_direccion           40 non-null     object 
 8   perfil_tipo                4000 non-null   object 
 9   perfil_otros               0 non-null      float64
 10  kpi_4_indice               4000 non-null   float64
 11  business_id                4040 non-null   object 
 12  latitude                   4040 non-null   Float64
 13  longitude                  4040 non-null   Float64
 14 

In [8]:
list_business_id = []
for i in range(min(data_random_powerbi.index),max(data_random_powerbi.index)+1):
    business_id = str(uuid.uuid4().hex)
    dict_case = {'index':i,'business_id':business_id}
    list_business_id.append(dict_case)
data_random_business_id = pd.DataFrame(list_business_id).set_index('index')